In [30]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig
from ast import literal_eval
from sklearn.model_selection import train_test_split



import os
import random
import numpy as np
import pandas as pd
import json

In [12]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42) # magic number :)

### 학습 데이터 불러오기

In [15]:
ROOT_DIR = '/data/ephemeral/pro-nlp-generationfornlp-nlp-13'
DATA_DIR = os.path.join(ROOT_DIR, 'data')
dataset = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             2031 non-null   object 
 1   paragraph      2031 non-null   object 
 2   problems       2031 non-null   object 
 3   question_plus  0 non-null      float64
dtypes: float64(1), object(3)
memory usage: 63.6+ KB


In [18]:
# Flatten the JSON dataset
records = []
for _, row in dataset.iterrows():
    problems = literal_eval(row['problems'])
    record = {
        'id': row['id'],
        'paragraph': row['paragraph'],
        'question': problems['question'],
        'choices': problems['choices'],
        'answer': problems.get('answer', None),
        "question_plus": problems.get('question_plus', None),
    }
    # Include 'question_plus' if it exists
    if 'question_plus' in problems:
        record['question_plus'] = problems['question_plus']
    records.append(record)
        
# Convert to DataFrame
df = pd.DataFrame(records)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2031 entries, 0 to 2030
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             2031 non-null   object
 1   paragraph      2031 non-null   object
 2   question       2031 non-null   object
 3   choices        2031 non-null   object
 4   answer         2031 non-null   int64 
 5   question_plus  0 non-null      object
dtypes: int64(1), object(5)
memory usage: 95.3+ KB


In [40]:
df.head()

,id,paragraph,question,choices,answer,question_plus,choices_len
0,generation-for-nlp-425,"상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服)...",상소한 인물이 속한 붕당에 대한 설명으로 옳은 것만을 모두 고르면?,"[ㄱ, ㄴ, ㄱ, ㄷ, ㄴ, ㄹ, ㄷ, ㄹ]",2,None,4
1,generation-for-nlp-426,"(가)은/는 의병계열과 애국계몽 운동 계열의 비밀결사가 모여 결성된 조직으로, 총사...",(가)에 대한 설명으로 옳지 않은 것은?,"[고려 문종 때에 남경(南京)으로 승격되었다., 종루(鐘樓), 이현, 칠패 등에서 ...",1,None,4
2,generation-for-nlp-427,나는 삼한(三韓) 산천의 음덕을 입어 대업을 이루었다.(가)는/은 수덕(水德)이 순...,(가) 지역에 대한 설명으로 옳은 것은?,"[이곳에 대장도감을 설치하여 재조대장경을 만들었다., 지눌이 이곳에서 수선사 결사운...",4,None,4
3,generation-for-nlp-428,이 날 소정방이 부총관 김인문 등과 함께 기 벌포에 도착하여 백제 군사와 마주쳤다....,밑줄 친 ‘그’에 대한 설명으로 옳은 것은?,"[살수에서 수의 군대를 물리쳤다 ., 김춘추 의 신라 왕위 계승을 지원하였다 ., ...",2,None,4
4,generation-for-nlp-429,"선비들 수만 명이 대궐 앞에 모여 만 동묘와 서원을 다시 설립할 것을 청하니, (가...",(가) 인물이 추진한 정책으로 옳지 않은 것은?,"[사창제를 실시하였다 ., 대전회통을 편찬하였다 ., 비변사의 기능을 강화하였다 ....",3,None,4


### Model Load

In [23]:
MODEL_NAME = "Qwen/Qwen3-8B"

tokenizer = AutoTokenizer.from_pretrained(
    "Qwen/Qwen3-8B"
    )
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen3-8B",
    torch_dtype=torch.float16,
    device_map="auto",
    )

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

### Data Preprocessing

In [29]:
df['choices_len'] = df['choices'].apply(len)

In [31]:
### 4/5로 분리
df_four = df[df['choices_len'] == 4].reset_index(drop=True)
df_five = df[df['choices_len'] == 5].reset_index(drop=True)

In [ ]:
four_train, four_val = train_test_split(
    df_four, test_size=0.1, random_state=42, shuffle=True
)

five_train, five_val = train_test_split(
    df_five, test_size=0.1, random_state=42, shuffle=True
)

In [35]:
ds_four_train = Dataset.from_pandas(four_train)
ds_four_val   = Dataset.from_pandas(four_val)

ds_five_train = Dataset.from_pandas(five_train)
ds_five_val   = Dataset.from_pandas(five_val)

In [36]:
ds_four_train

Dataset({
    features: ['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus', 'choices_len', '__index_level_0__'],
    num_rows: 712
})

In [39]:
SYSTEM_PROMPT = """당신은 '언어 이해' 및 '비문학 독해' 영역의 최상위권 전문가입니다.
주어진 [지문]의 내용을 절대적인 사실로 간주하고, 이를 바탕으로 [질문]에 대한 가장 적절한 답을 논리적으로 도출하십시오.
외부 지식이나 주관적인 추측은 철저히 배제하고, 오직 텍스트에 기반하여 정답을 선택하십시오.

[출력 형식]
- 반드시 한 줄로만 출력: "정답: X"
- X는 선택지 번호이며 1~5 중 하나의 정수
- 다른 어떤 설명/근거/문장도 출력하지 마십시오.
"""

In [ ]:
PROMPT_NO_QUESTION_PLUS = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 문제 해결 가이드라인
1. **질문 분석**: 질문이 긍정형(적절한 것)인지 부정형(적절하지 **않은** 것)인지 명확히 파악하십시오.
2. **근거 탐색**: 각 보기의 핵심 키워드가 지문의 어느 문장에 위치하는지 찾으십시오.
3. **사실 대조**: 지문의 내용과 보기의 내용을 1:1로 비교하여 일치 여부를 검증하십시오.
4. **오답 소거**: 지문의 내용과 다르거나(Contradiction), 지문에 언급되지 않은(Not Mentioned) 보기를 제거하십시오.
5. **결론 도출**: 남은 보기 중 가장 확실한 근거를 가진 번호를 정답으로 선택하십시오.

### 정답 번호
위 형식에 따라 "정답: X"만 출력하십시오.
"""


PROMPT_QUESTION_PLUS = """### 지문
{paragraph}

### 질문
{question}

### 선택지
{choices}

### 보기
{question_plus}

### 문제 해결 가이드라인
1. **질문 분석**: 질문이 긍정형(적절한 것)인지 부정형(적절하지 **않은** 것)인지 명확히 파악하십시오.
2. **근거 탐색**: 각 보기의 핵심 키워드가 지문의 어느 문장에 위치하는지 찾으십시오.
3. **사실 대조**: 지문의 내용과 보기의 내용을 1:1로 비교하여 일치 여부를 검증하십시오.
4. **오답 소거**: 지문의 내용과 다르거나(Contradiction), 지문에 언급되지 않은(Not Mentioned) 보기를 제거하십시오.
5. **결론 도출**: 남은 보기 중 가장 확실한 근거를 가진 번호를 정답으로 선택하십시오.

### 정답 번호
위 형식에 따라 "정답: X"만 출력하십시오.
"""

In [ ]:
def build_choices_string(choices):
    return "\n".join([f"{i+1}. {c}" for i, c in enumerate(choices)])


def is_missing(x):
    return x is None or x == "" or (isinstance(x, float) and math.isnan(x))

def make_messages_5(example):
    choices_str = build_choices_string(example["choices"])

    qp = example.get("question_plus", None)  # 컬럼 없으면 None

    if not is_missing(qp):
        user_msg = PROMPT_QUESTION_PLUS.format(   
            paragraph=example["paragraph"],
            question_plus=qp,
            question=example["question"],
            choices=choices_str,
        )
    else:
        user_msg = PROMPT_NO_QUESTION_PLUS.format( 
            paragraph=example["paragraph"],
            question=example["question"],
            choices=choices_str,
        )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_msg},
    ]

    ans = example.get("answer", None)
    if ans is not None and not (isinstance(ans, float) and math.isnan(ans)):
        messages.append({"role": "assistant", "content": f"정답: {int(ans)}"})

    return messages

In [73]:
def to_text(example):
    messages = make_messages_5(example)
    example["text"] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,  # 학습
        enable_thinking=False,        # 학습은 정답만 (추천)
    )
    return example

In [85]:
### 수정용
import re
THINK_RE = re.compile(r"<think>.*?</think>\s*", re.DOTALL)

def to_text(example):
    messages = make_messages_5(example)
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )
    text = THINK_RE.sub("", text)  # ✅ think 제거
    example["text"] = text
    return example

In [86]:
ds_five_train = Dataset.from_pandas(five_train)
ds_five_val   = Dataset.from_pandas(five_val)

train_ds = ds_five_train.map(to_text)
val_ds   = ds_five_val.map(to_text)

# 혹시 생기는 index 컬럼 제거
for col in ["__index_level_0__"]:
    if col in train_ds.column_names:
        train_ds = train_ds.remove_columns([col])
        val_ds   = val_ds.remove_columns([col])

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

In [87]:
train_ds[0]

{'id': 'generation-for-nlp-2853',
 'paragraph': '지난 27일 오후 6시 서울 불광동 ‘고양삼송 우남퍼스트빌’ 모델하우스. 저녁 시간인데도 방문객 행렬이 계속 이어졌다. 아이 손을 잡고 온 가족이 눈에 많이 띄었다. ‘4·1 부동산대책’ 이후 실수요자들이 서서히 내집 마련에 관심을 갖기 시작했다는 점을 느낄 수 있었다. 우남건설이 삼송지구의 명품 아파트를 표방하며 선보인 이 단지는 고객평가단 800여명의 의견을 반영해 세 차례나 설계를 바꿨다. 그래서인지 현관과 다용도실, 팬트리(식품저장소) 등 비주거공간을 최소화하는 대신 거실과 방을 널찍하게 확보한 게 눈에 들어왔다.○고객 평가단 800명 의견을 설계로모델하우스를 찾은 사람들은 소비자를 배려한 평면 설계에 좋은 반응을 보였다. 모든 주택형은 거실과 방 세 칸을 나란히 전면에 배치하는 4~4.5베이로 구성된다. 분양가에 포함되지 않는 서비스면적을 대폭 늘렸기 때문에 가능한 평면이다. 전용 64·74㎡의 서비스면적은 약 35㎡이고 84㎡B는 48㎡에 이른다. 자녀 방에는 수납 공간을 넉넉히 넣는 등 실속형 평면으로 꾸몄다. 벽지와 인테리어 등을 화이트 계열로 처리, 내부가 더 넓게 보였다. 이처럼 내부 공간을 넓게 설계할 수 있었던 건 우남건설이 침체된 시장 상황을 고려해 소비자의 의견을 끊임없이 반영했기 때문이다. 우남건설은 당초 전용 85㎡ 초과 부지인 이곳에 가구 수 변동 없이 소비자의 눈높이에 맞게 중소형 위주의 단지로 설계했다. 그 결과 내부 평면뿐 아니라 전반적인 단지 모양도 좋아졌다. 축구장 크기의 중앙광장이 조성되면서 동 간 거리가 70~90m로 넓어져 사생활 침해 요소가 줄었다. 녹지율은 대지 면적의 절반에 가까운 47%다. 각 동 1층(27가구)은 복층형 테라스하우스로 공급된다. 최상층 펜트하우스에서는 북한산이 보인다. 김종두 영업사업본부 실장은 “소비자의 의견을 청취하는 ‘인터렉티브(쌍방향 의사소통) 마케팅’을 펼쳐 실속 평면을 내놓게 됐다”며 “방문객들

In [84]:
print(train_ds[0]['text'])

<|im_start|>system
당신은 '언어 이해' 및 '비문학 독해' 영역의 최상위권 전문가입니다.
주어진 [지문]의 내용을 절대적인 사실로 간주하고, 이를 바탕으로 [질문]에 대한 가장 적절한 답을 논리적으로 도출하십시오.
외부 지식이나 주관적인 추측은 철저히 배제하고, 오직 텍스트에 기반하여 정답을 선택하십시오.

[출력 형식]
- 반드시 한 줄로만 출력: "정답: X"
- X는 선택지 번호이며 1~5 중 하나의 정수
- 다른 어떤 설명/근거/문장도 출력하지 마십시오.
<|im_end|>
<|im_start|>user
### 지문
지난 27일 오후 6시 서울 불광동 ‘고양삼송 우남퍼스트빌’ 모델하우스. 저녁 시간인데도 방문객 행렬이 계속 이어졌다. 아이 손을 잡고 온 가족이 눈에 많이 띄었다. ‘4·1 부동산대책’ 이후 실수요자들이 서서히 내집 마련에 관심을 갖기 시작했다는 점을 느낄 수 있었다. 우남건설이 삼송지구의 명품 아파트를 표방하며 선보인 이 단지는 고객평가단 800여명의 의견을 반영해 세 차례나 설계를 바꿨다. 그래서인지 현관과 다용도실, 팬트리(식품저장소) 등 비주거공간을 최소화하는 대신 거실과 방을 널찍하게 확보한 게 눈에 들어왔다.○고객 평가단 800명 의견을 설계로모델하우스를 찾은 사람들은 소비자를 배려한 평면 설계에 좋은 반응을 보였다. 모든 주택형은 거실과 방 세 칸을 나란히 전면에 배치하는 4~4.5베이로 구성된다. 분양가에 포함되지 않는 서비스면적을 대폭 늘렸기 때문에 가능한 평면이다. 전용 64·74㎡의 서비스면적은 약 35㎡이고 84㎡B는 48㎡에 이른다. 자녀 방에는 수납 공간을 넉넉히 넣는 등 실속형 평면으로 꾸몄다. 벽지와 인테리어 등을 화이트 계열로 처리, 내부가 더 넓게 보였다. 이처럼 내부 공간을 넓게 설계할 수 있었던 건 우남건설이 침체된 시장 상황을 고려해 소비자의 의견을 끊임없이 반영했기 때문이다. 우남건설은 당초 전용 85㎡ 초과 부지인 이곳에 가구 수 변동 없이 소비자의 눈높이에 맞게 중소형 위주의 단지